<a href="https://colab.research.google.com/github/viresnet/viresnet/blob/master/ViResNet_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


---
# Training of ViresNet


In [ ]:
#@title Standart imports for ResNet50 training
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D,Dropout,Flatten,Dense,Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
#@title Load ViResNet library


In [ ]:
#@title Intialize data and augmentation pipelines

#Initialize data path 
basePath = 'BASE_PATH' 

# Initilize training,validation and testing paths
trainPath = os.path.sep.join([basePath, "train"])
validPath = os.path.sep.join([basePath, "valid"])
testPath = os.path.sep.join([basePath, "test"])

#######################
# count the total number of imagesin training, validation,

totalTrain = listFilesAndCount(trainPath)[0]
totalVal = listFilesAndCount(trainPath)[0]
totalTest = listFilesAndCount(trainPath)[0]

#intialize training params
batchSize = 32
targetSize=(224, 224)
numEpochs = 1000

# initialize the training generator with shifts,shears,rotations etc.
trainGen = trainAug.flow_from_directory(
	rotation_range=25,
	zoom_range=0.1,
	width_shift_range=0.1,
	height_shift_range=0.1,
	shear_range=0.2,
	horizontal_flip=True,
	fill_mode="nearest")

# initialize the validation generator
valGen = valAug.flow_from_directory(
	validPath,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=False,
	batch_size=batchSize)

# initialize the testing generator
testGen = valAug.flow_from_directory(
	testPath,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=False,
	batch_size=batchSize)






In [ ]:
#@title Construction of the model

# load the ResNet-50 network, ensuring the head FC layer sets are left
# off
baseModel = ResNet50(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

# Construct a custom head of the model that will be placed on top of the base
#Resnet50 model. Several dense layers could be added if no convergence is reached
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(1024, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

# Join the head and the base
model = Model(inputs=baseModel.input, outputs=headModel)

#in order to use pretrained base model as a starting point we first freeze all layers
#but the head ones
for layer in baseModel.layers:
	layer.trainable = False



In [ ]:
#@title Training of the model

#first we compile the model by initializing an Adam optimizer with low learning rate
opt = Adam(lr=1e-5, decay=1e-5 / numEpochs)
#during compiling of the model the loss function has to be changed to categorical 
#cross entropy if more than 2 classes are present
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])


#train the model
History = model.fit_generator(
	trainGen,
	steps_per_epoch=totalTrain // batchSize,
	validation_data=valGen,
	validation_steps=totalVal // batchSize,
	epochs=numEpochs)
#in order to fine tune the results further, one can subsequently layer by 
#layer unfreeze the base model

#look through the layers
for i, layer in enumerate(model.layers):
   print(i, layer.name, layer.trainable)
#enable layers for training
for layer in conv_base.layers[165:]:
   layer.trainable = True

#save the fine tuned model
model.save('fineTunedViResNet.h5')
